In [2]:
#track test
import sys, os
parent_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)))
#print(path2add)
sys.path.append(parent_path)
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","underpinnings")))
from notebook_log_loading import load_sensor_log_file
from MessageToJSON import json_to_message
from id_to_name import id_to_name

#sensor_log,first_timestamp=load_sensor_log_file("../to_study/sound_study.txt")

In [ ]:
import numpy as np
import torch
import torchaudio



melspectogram_transform = torchaudio.transforms.MelSpectrogram(
  sample_rate=16000, n_mels=128)
melspectogram_db_transform = torchaudio.transforms.AmplitudeToDB()

In [3]:
import matplotlib.pyplot as plt

from os import walk
import wave
p = pyaudio.PyAudio()


mypath="sounds/"
filenames = next(walk(mypath), (None, None, []))[2]
print("files are: {}".format(filenames))
max_chunk_size=16000*5 # 5 seconds

for f in filenames:
    wf = wave.open(f, 'rb')
    data = wf.readframes(max_chunk_size)
    wf.close()
    data = array.array('h', data)
    #turn into pytorch array
    data=torch.array(data)
    plt.figure()
    plt.plot(data[:])
    #turn into mel spectrograms
    melspectrogram=melspectogram_transform(data)
    plt.figure()
    plt.imshow(melspectogram.squeeze().numpy(), cmap='hot')
 
    melspectogram_db=melspectogram_db_transform(melspectogram)
    plt.figure()
    plt.imshow(melspectogram_db.squeeze().numpy(), cmap='hot')




files are: []
